In [1]:
import numpy as np
import scipy.linalg
import scipy.special
import math
import sys
import matplotlib.pyplot as plt
import json

# Quartic EM Response Part 2: Calculate coefficients c_p

In this second notebook, we import the eigenvalues and eiegenvectors computed previously, and use them to calculate the current per orbital

### Coefficients

In [2]:
# These coefficients are defined in the tex document 'quartic EM Response' ('neat' version is best)

# n is in the definition of H^{2n}
# p is the prefactor to the coefficient c_p

# When dtilde is true (default), this uses David's definition of the Hamiltonians
def dcoeff(n, b, c, dtilde=True):
    if(dtilde):
        return (scipy.special.factorial(n-1)/scipy.special.factorial(b)/scipy.special.factorial(c) 
                * scipy.special.factorial2(2*n-1)/scipy.special.factorial2(2*n-b-c))
    else:
        return (2*scipy.special.factorial(n)/scipy.special.factorial(b)/scipy.special.factorial(c) 
                * scipy.special.factorial2(2*n-1)/scipy.special.factorial2(2*n-b-c))
    
def fcoeff(p, b, c):
    return (scipy.special.factorial(p)/scipy.special.factorial(b)/scipy.special.factorial(c)
            /scipy.special.factorial2(p-b-c))

def acoeff(t, b, c):
    if(t>=c):
        return np.sqrt(scipy.special.factorial(t)*scipy.special.factorial(t-c+b))/scipy.special.factorial(t-c)
    else:
        return 0
    
def bacterm(lam, b, c, t, coeff_matrix):
    if(b!=0):
        return b*acoeff(t,b-1,c)*np.conjugate(coeff_matrix[:,lam][t-c+b-1])
    else:
        return 0

def cacterm(lam, b, c, t, coeff_matrix):
    if(c!=0):
        return c*acoeff(t,b,c-1)*np.conjugate(coeff_matrix[:,lam][t-c+1+b])
    else:
        return 0



### Summand

In [3]:
# This is the main expression to calculate and sum over all indices

def summand(n, mu, lam, p, b, c, bet, gam, t, tau, bval, coeff_matrix, 
            energy_list, dtilde=True):
    coeff_matrix = np.array(coeff_matrix)
    energy_list = np.array(energy_list)
    return (-(bval**n) * (np.sqrt(2*bval) ** (-p-1)) * coeff_matrix[:,mu][t] *
            np.conjugate(coeff_matrix[:,lam][tau-gam+bet]) * coeff_matrix[:,mu][tau] *
            dcoeff(n, b, c, dtilde) * fcoeff(p, bet, gam) * acoeff(tau, bet, gam) /
            (energy_list[lam] - energy_list[mu]) * (bacterm(lam, b, c, t, coeff_matrix)
                                                    + cacterm(lam, b, c, t, coeff_matrix)))
            


### Index Lists

In [4]:
# These functions generate the lists of indices to be summed over

def mulist(lam, mu_max):
    list_out = np.array([j for j in range(mu_max+1)])
    list_out = np.delete(list_out, lam)
    return list_out

def tlist(t_max):
    return np.array([j for j in range(t_max+1)])

def betgamlist(p):
    list_out = []
    for b in range(p+1):
        for c in range(p+1-b):
            if(((b+c)%2)==(p%2)):
                list_out.append([b,c])
    return list_out

def bclist(n):
    list_out = []
    for b in range(2*n+1):
        for c in range(2*n+1-b):
            if(((b-c)%4)==0):
                list_out.append([b,c])
    return list_out
                


## Calculate Current Per Orbital
### Import Data From File

In [5]:
def read_evals_evecs(n, trunc_dim):
    
    filename_evals = "n"+str(n)+"_truncdim"+str(trunc_dim)+"_evals"
    filename_evecs = "n"+str(n)+"_truncdim"+str(trunc_dim)+"_evecs"
    
    with open(filename_evals) as f:
        q_evals = json.load(f)
        
    with open(filename_evecs) as f:
        q_evecs = json.load(f)
    
    return q_evals, q_evecs

### Calculate EM Response

In [6]:
def cp_coefficient_lambda(n, p, lam, coeff_matrix, energy_list, bval=1, mu_max=15, t_max=25, tol=1e-12, dtilde=True):
    
    this_tlist = tlist(t_max)
    this_betgamlist = betgamlist(p)
    this_bclist = bclist(n)
    this_mu_list = mulist(lam, mu_max)
    
    sum_total = 0
        
    for mu in this_mu_list:
        for tau in this_tlist:
            for t in this_tlist:
                for [b,c] in this_bclist:
                    for [bet, gam] in this_betgamlist:
                        this_element = summand(n, mu, lam, p, b, c, bet, gam, t, tau, bval,
                                                coeff_matrix, energy_list, dtilde=True)
                        if np.abs(this_element) > tol:
                            sum_total += this_element
                            
    return sum_total

In [7]:
def cp_coefficient_lambda_loud(n, p, lam, coeff_matrix, energy_list, bval=1, mu_max=15, t_max=25, tol=1e-12, dtilde=True):
    
    this_tlist = tlist(t_max)
    this_betgamlist = betgamlist(p)
    this_bclist = bclist(n)
    this_mu_list = mulist(lam, mu_max)
    
    sum_total = 0
        
    for mu in this_mu_list:
        for tau in this_tlist:
            for t in this_tlist:
                for [b,c] in this_bclist:
                    for [bet, gam] in this_betgamlist:
                        this_element = summand(n, mu, lam, p, b, c, bet, gam, t, tau, bval,
                                                coeff_matrix, energy_list, dtilde=True)
                        if np.abs(this_element) > tol:
                            if np.abs(this_element) > 1e-2:
                                print('mu =', mu, ", t =",t, ", b =", b, ", c =", c, ", beta =", bet, ", gamma =", gam)
                                print(this_element)
                            sum_total += this_element
                            
    print('Total =', sum_total)
    return sum_total

In [20]:
def cp_coefficient_table(n, p_max, lam_max, trunc_dim=50, bval=1, mu_max=15, t_max=25, tol=1e-12, dtilde=True):
    
    table_out = []
    elist, coeffmat = read_evals_evecs(n, trunc_dim)
    
    for p in range(1,p_max+1,2):
        print('Starting p =', p)
        row_out = []
        for lam in range(lam_max+1):
            row_out.append(
                cp_coefficient_lambda(n, p, lam, coeffmat, elist, bval, mu_max, t_max, tol, dtilde)
            )
        table_out.append(row_out)
    
    filename_EM = ("n"+str(n)+"_truncdim"+str(trunc_dim)+"_pmax"+str(p_max)+"_lam_max"+str(lam_max)
                   +"_mu_max"+str(mu_max)+"_t_max"+str(t_max)+"_cp_table")
    
    with open(filename_EM, "w") as f:
        json.dump(table_out, f)
        
    print("Complete.")
    
    #print(table_out)
    

### Check for LL again

In [19]:
cp_coefficient_table(1,5,3,trunc_dim=50,bval=1,mu_max=5,t_max=10)

Starting p = 1
Starting p = 3
Starting p = 5


In [21]:
cp_coefficient_table(2,5,3)

Starting p = 1
Starting p = 3
Starting p = 5
Complete.


In [22]:
cp_coefficient_table(3,5,3)

Starting p = 1
Starting p = 3
Starting p = 5
Complete.


In [ ]:
# slow check
cp_coefficient_table(2, 5, 3, trunc_dim=100, bval=1, mu_max=40, t_max=60, tol=1e-12, dtilde=True)

Starting p = 1
